# Kaggle: Netflix Movies and TV shows

Author: Katherine Zhang 

Start date: April, 6, 2020

**Background**: This dataset consists of tv shows and movies available on Netflix as of 2020. The dataset is collected from Flixable which is a third-party Netflix search engine. The data can be downloaded from https://www.kaggle.com/shivamb/netflix-shows.

**Motivation:**

- Understanding what content is available in different countries
- Determeine whether Netflix has increasingly focusing on TV rather than movies in recent years.
- Identifying similar content by matching text-based features


In [0]:
import nltk
import re
import pandas as pd
import numpy as np
import seaborn as sns 
import collections as c
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
# retrieves dataset 
%%capture 
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pw_JghkssxSeodE_2IlmpHUvtjW0IKPQ' -O 'netflix.csv'

### **0. Initial Checking**

In [173]:
df_netflix = pd.read_csv('netflix.csv')
df_netflix.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."


In [174]:
len(df_netflix.drop_duplicates())

6234

In [175]:
len(df_netflix[df_netflix['type'] == 'Movie'])

4265

There are 6234 movies and tv shows on netflix. About 68% of them are movies and 32% are TV shows. 

## **2. EDA**

- **Data Preparation** 

In [176]:
df_netflix['rating'] = df_netflix['rating'].fillna("")
df_netflix['date_added'] = pd.to_datetime(df_netflix['date_added'])
df_netflix['year_added'] = df_netflix['date_added'].dt.year
list(df_netflix[df_netflix['date_added'].isna()]['title'])

['Gunslinger Girl',
 'Anthony Bourdain: Parts Unknown',
 'Frasier',
 'La Familia P. Luche',
 'The Adventures of Figaro Pho',
 'Kikoriki',
 'Red vs. Blue',
 'Maron',
 'Little Baby Bum: Nursery Rhyme Friends',
 "A Young Doctor's Notebook and Other Stories",
 'Friends']

There're 11 shows with NA values in 'date_added' and further research shows that they are not available on Netflix anymore. Ex. 'Friends' left Netflix in 2020. We will drop those shows which are no longer available on Netflix. 

In [0]:
df_netflix = df_netflix[df_netflix['date_added'].notna()]
df_netflix = df_netflix[df_netflix['title'].notna()]

- **Objective 1: Understanding what content is available in different countries**

Currently Netflix is available in 113 countries and regions. Let's see which country has the most content. 

In [178]:
country_count = c.Counter(", ".join(df_netflix['country'].dropna()).split(", "))
top_ten_countries = country_count.most_common(10)
country = [val[0] for val in top_ten_countries][::-1]
show_count = [val[1] for val in top_ten_countries][::-1]
trace1 = go.Bar(y=country, x=show_count, orientation="h", name="", marker=dict(color='#3498DB'))
data = [trace1]
layout = go.Layout(title="Top 10 countries with most content on Netflix", height=400, width=700, legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

Let's also look at the top genre on Netflix.

In [179]:
top_ten_genre = c.Counter((", ".join(df_netflix['listed_in'])).split(", ")).most_common(10)
genre = [val[0] for val in top_ten_genre][::-1]
count = [val[1] for val in top_ten_genre][::-1]
trace2 = go.Bar(y=genre, x=count, orientation="h", name="", marker=dict(color='#3498DB'))
data = [trace2]
layout = go.Layout(title="Top 10 genres on Netflix", height=400, width=700, legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

In [0]:
def df_by_country(df, country):
  '''
  Returns a filtered df with shows available to a specific country
  Input: A dataframe and a selected country name
  Output: A dictionary with country as key and show_id as value
  '''
  drop_country_na = df[df['country'].notna()]
  return drop_country_na[drop_country_na['country'].str.contains(country)]

In [181]:
def top_genre_by_country(df, country):
  genre_counter = c.Counter(", ".join(df_by_country(df_netflix, country)['listed_in']).split(", ")).most_common(10)
  genre = [val[0] for val in genre_counter][::-1]
  count = [val[1] for val in genre_counter][::-1]
  return genre, count

fig = make_subplots(rows=2, cols=2, horizontal_spacing=0.4,
      subplot_titles=("USA",'France', 'Japan', 'South Korea'))
country = ['United States', 'France', 'Japan', 'South Korea']
colors = ['#AF7AC5', '#76D7C4', '#EC7063', '#F4D03F']
position = [(1,1), (1,2), (2,1), (2,2)]

for i in np.arange(len(country)):
  genre, count = top_genre_by_country(df_netflix, country[i])
  fig.add_trace(go.Bar(y=genre, x=count, 
                       orientation="h", name="", 
                       marker=dict(color=colors[i])), position[i][0],  position[i][1])
  
  fig.update_layout(showlegend=False, height = 650, width = 900, title_text="Top 5 genres by country")
  
fig.show()

As we observe from the barplots above, the top 10 genre of Netflix shows vary from country to country, which may reflect the difference in the tastes of audience. For example, American and French viewers on Netflix may favor `Drama` and `Comedies` over `Anime series` compared to Japanese viewers, whereas viewers in South Korea favor `Korean TV shows` the most.

-  **Objective 2) Determine if Netflix has increasingly focusing on TV rather than movies in recent years.**

In [182]:
tv_add_count = df_netflix[df_netflix['type'] == 'TV Show'].groupby('year_added').size().reset_index(name = 'added_count').iloc[0:9,:]
movie_add_count = df_netflix[df_netflix['type'] == 'Movie'].groupby('year_added').size().reset_index(name = 'added_count').iloc[0:12,:]
trace1 = go.Scatter(x=tv_add_count['year_added'], y=tv_add_count['added_count'], name="TV Shows", marker=dict(color="#3498DB"))
trace2 = go.Scatter(x=movie_add_count["year_added"], y=movie_add_count['added_count'], name="Movies", marker=dict(color="#EC7063"))
data = [trace1, trace2]
layout = go.Layout(title="TV shows vs. Movies added over the years", width = 600, legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

We observe that the number of movies added to Netflix  is higher than that of TV shows all time. In 2019, Netflix added 1546 movies and 803 TV shows. So there is no evidence that Netflix has switched focus from movies to TV shows.  

## **3. Content-based recommendation of Netflix shows**

We want to find the top 5 similar shows given the following text-based feature of a list of shows. 
 
- `Description`
- `Listed in`
- `Rating`



*   **Data preparation** 



In [0]:
df_netflix['aggregated_text'] = df_netflix['description'] + " " + " " + df_netflix['listed_in'].str.lower() + " " + df_netflix['rating'].str.lower()          
corpus_tokenized = list(df_netflix['aggregated_text'].str.split(" "))
stopwords_list = set(stopwords.words("english"))
index = list(range(0, len(corpus_tokenized)))
clean_corpus = []

for sentence in corpus_tokenized:
  s = []
  for word in sentence:
    clean_word = re.sub(r'[^\w\s]','', word)
    if clean_word not in stopwords_list:
      s.append(clean_word)
  clean_corpus.append(" ".join(s))

In [0]:
def get_recommendation(show_list, vectorizer):
  '''
  Returns: 
        A df showing the top 5 similar show and genres of a given list of shows 
  Input: 
        Show_name: A list of shows that the user has already watched
        Vectorizer: Type of vectorizer 
  '''
  title, scores, genre = [], [], []
  for show_name in show_list:
    show_index = df_netflix[df_netflix['title'] == show_name].index[0]
    cosine_similarities = linear_kernel(vectorizer[show_index], vectorizer).flatten()
    similar_show_index = cosine_similarities.argsort()[:-7:-1][1:]
    title += [df_netflix['title'][i] for i in similar_show_index]
    genre += [df_netflix['listed_in'][i] for i in similar_show_index]
    scores += list(cosine_similarities[similar_show_index])

  df = pd.DataFrame(data = {'Title': title, 
                            'Genre': genre,
                            'Cosine_similarity': scores})
  df = df[~df['Title'].isin(show_list)].sort_values('Cosine_similarity', ascending = False)
  df['Title'] = df['Title'].drop_duplicates()
  return df[df['Title'].notna()].iloc[0:5, :]

- **TF-IDF vectorizer**

In [338]:
tfidf_vectorizer = TfidfVectorizer().fit_transform(clean_corpus)
watched_shows = ['Stranger Things', 'The Vampire Diaries', 'Sense8']
get_recommendation(watched_shows, tfidf_vectorizer)

,Title,Genre,Cosine_similarity
0,Once Upon a Time,"TV Dramas, TV Sci-Fi & Fantasy",0.231036
1,The OA,"TV Dramas, TV Mysteries, TV Sci-Fi & Fantasy",0.221062
10,The Matrix Revolutions,"Action & Adventure, Sci-Fi & Fantasy",0.220382
2,Anjaan: Special Crimes Unit,"International TV Shows, TV Horror, TV Mysteries",0.212523
11,Dragons: Race to the Edge,"Kids' TV, TV Action & Adventure",0.211031


Next, we want to look at how vectorizer performs differently when use **both unigram and bigram**.

In [339]:
tfidf_vectorizer = TfidfVectorizer(ngram_range = (1, 2)).fit_transform(clean_corpus)
get_recommendation(watched_shows, tfidf_vectorizer)

,Title,Genre,Cosine_similarity
10,The OA,"TV Dramas, TV Mysteries, TV Sci-Fi & Fantasy",0.160266
0,Chilling Adventures of Sabrina,"TV Horror, TV Mysteries, TV Sci-Fi & Fantasy",0.152653
5,The 4400,"TV Dramas, TV Mysteries, TV Sci-Fi & Fantasy",0.142792
1,Nightflyers,"TV Horror, TV Mysteries, TV Sci-Fi & Fantasy",0.142747
12,The Messengers,"TV Dramas, TV Mysteries, TV Sci-Fi & Fantasy",0.142091


We observe that the cosine similarity is now lower than the vectorizer using **only unigram**, and the `Genre` of the shows become more similar(almost identical) to the chosen show. 